In [1]:
%cd /content/drive/MyDrive/DSS/project/data

/content/drive/MyDrive/DSS/project/data


In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('/content/drive/MyDrive/DSS/project/data/database2/Ratings.csv', 
                      dtype={'movie_id': int, 'user_id':int}, nrows=1000000)
ratings.dropna(how='any', inplace=True)
ratings.drop_duplicates(inplace=True)
ratings

,user_id,movie_id,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1246,5.0,1425941556
4,1,1968,4.0,1425942148
...,...,...,...,...
996462,23624,1358,3.5,1071466216
996463,23624,1392,4.0,1071467036
996464,23624,1394,4.0,1071466140
996465,23624,1914,3.5,1071469437


In [5]:
!ls

data  database	database2  process_data_2.ipynb  processDataForDatabase.ipynb


In [6]:
ratings.to_csv('./database2/Ratings.csv', index=False)

In [ ]:
def convert_id(val):
    try:
        return int(val)
    except:
        return -1
movie_metadata = pd.read_csv('./data/movies_metadata.csv',
                             usecols=['belongs_to_collection',
                                       'id',
                                       'genres', 
                                       'imdb_id', 
                                       'title',
                                       'overview', 
                                       'release_date'],
                              converters={'id': convert_id})
movie_metadata

,belongs_to_collection,genres,id,imdb_id,overview,release_date,title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II
...,...,...,...,...,...,...,...
45461,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,tt6209470,Rising and falling between a man and woman.,NaN,Subdue
45462,NaN,"[{'id': 18, 'name': 'Drama'}]",111109,tt2028550,An artist struggles to finish his work while a...,2011-11-17,Century of Birthing
45463,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,tt0303758,"When one of her hits goes wrong, a professiona...",2003-08-01,Betrayal
45464,NaN,[],227506,tt0008536,"In a small town live two brothers, one a minis...",1917-10-21,Satan Triumphant


In [ ]:
# drop NA
movie_metadata.dropna(axis=0, subset=['id',
                                       'genres', 
                                       'imdb_id', 
                                       'title',
                                       'overview', 
                                       'release_date'], 
                      how='any',inplace=True)
# drop duplicates
movie_metadata.drop_duplicates(inplace=True)
movie_metadata

,belongs_to_collection,genres,id,imdb_id,overview,release_date,title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II
...,...,...,...,...,...,...,...
45460,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,tt0102797,"Yet another version of the classic epic, with ...",1991-05-13,Robin Hood
45462,NaN,"[{'id': 18, 'name': 'Drama'}]",111109,tt2028550,An artist struggles to finish his work while a...,2011-11-17,Century of Birthing
45463,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,tt0303758,"When one of her hits goes wrong, a professiona...",2003-08-01,Betrayal
45464,NaN,[],227506,tt0008536,"In a small town live two brothers, one a minis...",1917-10-21,Satan Triumphant


In [ ]:
import ast
import math

In [ ]:
# create the poster path column
poster_paths = []
for c in movie_metadata['belongs_to_collection']:
    try:
        d = ast.literal_eval(c)
        poster_paths.append(d['poster_path'])
    except:
        poster_paths.append('nan')

In [ ]:
movie_metadata['poster_paths'] = poster_paths

# Drop ratings of inexist movies

In [ ]:
inexist_movie_ids = set(ratings['movie_id']).difference(movie_metadata['id'])
inexist_movie_indexes = []
for i in list(inexist_movie_ids):
    # append a index list
    inexist_movie_indexes.extend(ratings[ratings['movie_id'] == i].index)
ratings.drop(index=inexist_movie_indexes, inplace=True)
ratings

,user_id,movie_id,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1246,5.0,1425941556
4,1,1968,4.0,1425942148
...,...,...,...,...
999995,23624,1358,3.5,1071466216
999996,23624,1392,4.0,1071467036
999997,23624,1394,4.0,1071466140
999998,23624,1914,3.5,1071469437


# Drop movies that do not exist in ratings

In [ ]:
# movie ids that only exist in ratings
inexist_movie_ids = set(movie_metadata['id']).difference(set(ratings['movie_id']))
inexist_movie_indexes = []
for i in list(inexist_movie_ids):
    # append a index list
    inexist_movie_indexes.extend(movie_metadata[movie_metadata['id'] == i].index)
movie_metadata.drop(index=inexist_movie_indexes, inplace=True)

In [ ]:
set(movie_metadata['id']).difference(set(ratings['movie_id']))

set()

# Drop inexist movies in MovieGenre.csv

In [ ]:
!pwd

/content/drive/MyDrive/DSS/project/data


In [ ]:
movie_genre = pd.read_csv('./database2/MovieGenres.csv', 
                          dtype={'movie_id': int})
movie_genre

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
90924,30840,10749
90925,111109,18
90926,67758,28
90927,67758,18


In [ ]:
# movie ids that only exist in ratings
inexist_movie_ids = set(movie_genre['movie_id']).difference(set(ratings['movie_id']))
inexist_movie_indexes = []
for i in list(inexist_movie_ids):
    # append a index list
    inexist_movie_indexes.extend(movie_genre[movie_genre['movie_id'] == i].index)
movie_genre.drop(index=inexist_movie_indexes, inplace=True)

In [ ]:
set(movie_genre['movie_id']).difference(set(ratings['movie_id']))

set()

#Create user table from ratings

In [ ]:
user = pd.DataFrame(data={
    'user_id': list(set(ratings['user_id'])),
    'user_name': list(set(ratings['user_id'])),
    'password': list(set(ratings['user_id']))
})
user

,user_id,user_name,password
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
...,...,...,...
23177,23620,23620,23620
23178,23621,23621,23621
23179,23622,23622,23622
23180,23623,23623,23623


# Save to files

In [ ]:
ratings

,user_id,movie_id,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1246,5.0,1425941556
4,1,1968,4.0,1425942148
...,...,...,...,...
999995,23624,1358,3.5,1071466216
999996,23624,1392,4.0,1071467036
999997,23624,1394,4.0,1071466140
999998,23624,1914,3.5,1071469437


In [ ]:
movie_metadata.columns

Index(['belongs_to_collection', 'genres', 'id', 'imdb_id', 'overview',
       'release_date', 'title', 'poster_paths'],
      dtype='object')

In [ ]:
movie = pd.DataFrame(data={
    'movie_id' : movie_metadata['id'].to_list(),
    'imdb_id' : movie_metadata['imdb_id'].to_list(),
    'title' : movie_metadata['title'].to_list(),
    'overview' : movie_metadata['overview'].to_list(),
    'release_date' : movie_metadata['release_date'].to_list(),
    'poster_paths' : movie_metadata['poster_paths'].to_list()
})
movie

,movie_id,imdb_id,title,overview,release_date,poster_paths
0,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg
1,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15,nan
2,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",1995-12-15,nan
3,710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,1995-11-16,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg
4,1408,tt0112760,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",1995-12-22,nan
...,...,...,...,...,...,...
5537,64197,tt1047845,Travelling with Pets,Plucked from an orphanage as a literal love sl...,2007-06-25,nan
5538,98604,tt2147597,Cinderella,"Masha Krapivina - is yet beautiful, and not th...",2012-02-14,nan
5539,5589,tt0232750,Šíleně smutná princezna,No overview found.,1968-06-07,nan
5540,49280,tt0135453,The One-Man Band,A band-leader has arranged seven chairs for th...,1900-01-01,nan


In [ ]:
movie_genre

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
90875,5589,10751
90904,49280,14
90905,49280,28
90906,49280,53


In [ ]:
user.to_csv('database2/Users.csv', index=False)

In [ ]:
movie.to_csv('database2/Movies.csv', index=False)
ratings.to_csv('database2/Ratings.csv', index=False)

In [ ]:
movie_genre.to_csv('database2/MovieGenres.csv', index=False)